In [662]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.layers import LSTM
from keras.layers import GRU
from keras import regularizers
import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import seaborn as sn
import itertools  
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
import random
from sklearn.utils import resample

In [663]:
#Function to print confusion matrix in a nice format.
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [664]:
##Specifying training data parameters.
years=range(2000,2019)
#years=range(2000,2010)
num_years=10
#Features/columns that will be used for prediction.
#features=["count","neighborCountyAvg", "Gini", "Temp", "Prec", "Hum", "County_type", "Incident_nominal", Resident_population_White_alone_percent", "Median_Household_Income", "Poverty_percent_of_people"]
features=["count","neighborCountyAvg", "Gini", "Temp", "Prec", "Hum", "County_type","Incident_nominal"]

In [665]:
#Importing training and testing data
train_path="/Users/sparshagarwal/Desktop/NCSA/Dataframes/Temporal_df.csv"
temporal_df=pd.read_csv(train_path)
test_path="/Users/sparshagarwal/Desktop/NCSA/Dataframes/Temporal_df_test.csv"
temporal_df_test=pd.read_csv(test_path)

In [666]:
temporal_df.head()

,A,B,C,D,E,F,G,H,I,J,...,6,7,8,9,10,11,12,13,14,15
0,1,0.000000,0.544365,292.763352,3.006250,72.683919,1,1,0,0.2,...,0,0,0,0,0,0,0,0,0,0
1,2,0.666667,0.529595,293.719919,4.002598,78.128852,1,1,0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,0,0.000000,0.510778,293.550745,2.879374,72.633073,0,1,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0,0.500000,0.541940,292.541822,2.947005,73.039254,0,1,1,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0,0.333333,0.555216,291.308402,2.702201,70.986648,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [667]:
#Checking frequency of instances for each class
dict_freq={}
for i in range(1,16):
    dict_freq[i]=len(temporal_df[temporal_df[str(i)]==1])
dict_freq

{1: 21156,
 2: 3282,
 3: 222,
 4: 59,
 5: 25,
 6: 19,
 7: 7,
 8: 7,
 9: 2,
 10: 4,
 11: 3,
 12: 11,
 13: 6,
 14: 3,
 15: 2}

In [668]:
#Oversampling to balance the classes
num_cl=len(temporal_df[temporal_df["1"]==1])
df_balanced=temporal_df[temporal_df["1"]==1]
for i in range(2,16):
    df_temp=temporal_df[temporal_df[str(i)]==1]
    if(len(df_temp)>0):
        df_minority_upsampled = resample(df_temp, replace=True, n_samples=num_cl, random_state=4)
        df_balanced=pd.concat([df_balanced, df_minority_upsampled])
df_balanced = df_balanced.sample(frac = 1, random_state=4)   #Shuffling the data
X_pre = df_balanced.iloc[:, 0:(num_years*len(features))]
Y_pre = df_balanced.iloc[:, (num_years*len(features)):]

In [669]:
# #Using unbalanced data
# temporal_df = temporal_df.sample(frac = 1) #Shuffling the data
# X_pre = temporal_df.iloc[:, 0:(num_years*len(features))]
# Y_pre = temporal_df.iloc[:, (num_years*len(features)):]

In [670]:
X=X_pre.values
Y=Y_pre.values

In [671]:
# #Balancing the data w.r.t. class labels
# ros = RandomOverSampler()
# X, Y = ros.fit_resample(X_pre.values,Y_pre.values)

In [672]:
#Shows all the class labels are now equally represented
y=np.argmax(Y, axis=1)
np.bincount(y)

array([21156, 21156, 21156, 21156, 21156, 21156, 21156, 21156, 21156,
       21156, 21156, 21156, 21156, 21156, 21156])

In [673]:
# # Calculating classs weights based on their frequencies
# y=np.argmax(temporal_df.iloc[:,-15:].values, axis=1)
# class_weights=compute_class_weight("balanced", [i for i in range(15)], y)

## LSTM Implementation for classification

In [674]:
def create_network():
    network = Sequential()
    network.add(BatchNormalization(input_shape=(num_years, len(features))))
    network.add(Dense(4, activation="tanh"))
    network.add(LSTM(15, dropout = 0.2, recurrent_dropout = 0.2, activation="tanh"))
    network.add(Dense(15, activation="softmax"))
    network.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return network

In [675]:
# #Cross-validation setup
# acc_train=[]
# acc_val=[]
# kf = KFold(n_splits=5, shuffle=True)
# for train_index, val_index in kf.split(X):   
#     X_train=X[train_index]
#     Y_train=Y[train_index]
#     X_val=X[val_index]
#     Y_val=Y[val_index]
    
#     #Transforming input variables into LSTM input format
#     X_train = X_train.reshape(X_train.shape[0], num_years, len(features))
#     X_val = X_val.reshape(X_val.shape[0], num_years, len(features))
#     Y_train=Y_train
#     Y_val=Y_val
    
#     #Creating model
#     model=create_network()
#     Hist=model.fit(X_train, Y_train, epochs=10, validation_data=(X_val, Y_val), verbose=2, class_weight=None)
    
#     #Final epoch accuracies for training and validation dataset
#     acc_train.append(Hist.history["accuracy"][-1])
#     acc_val.append(Hist.history["val_accuracy"][-1])
    
# print("Training accuracy:" + str(np.mean(acc_train)))
# print("Validation accuracy:" + str(np.mean(acc_val)))
    

In [676]:
# Dataset while using test_train split for final prediction

#Transforming input variables into LSTM input format

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.20 , random_state=4)

X_train = X_train.reshape(X_train.shape[0], num_years, len(features))
X_val = X_val.reshape(X_val.shape[0], num_years, len(features))
Y_train=Y_train
Y_val=Y_val

X_test=temporal_df_test.iloc[:, 0:(num_years*len(features))]
X_test = X_test.values.reshape(X_test.shape[0], num_years, len(features))

In [677]:
model=create_network()

In [678]:
Hist=model.fit(X_train, Y_train, nb_epoch=30, validation_data=(X_val, Y_val), verbose=2, class_weight=None)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 253872 samples, validate on 63468 samples
Epoch 1/30


KeyboardInterrupt: 

In [ ]:
# Plot training & validation accuracy values
plt.plot(Hist.history['accuracy'])
plt.plot(Hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(Hist.history['loss'])
plt.plot(Hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# #Using unbalanced data
# temporal_df = temporal_df.sample(frac = 1) #Shuffling the data
# X_pre = temporal_df.iloc[:, 0:(num_years*len(features))]
# Y_pre = temporal_df.iloc[:, (num_years*len(features)):]
# X=X_pre.values
# Y=Y_pre.values
# X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.20)

# X_train = X_train.reshape(X_train.shape[0], num_years, len(features))
# X_val = X_val.reshape(X_val.shape[0], num_years, len(features))
# Y_train=Y_train
# Y_val=Y_val

In [ ]:
#Predicting value for train, val, and test datasets
pred_train=model.predict(X_train)
pred_val=model.predict(X_val)
#pred_test=model.predict(X_test)

In [ ]:
#Converting probabilities to class labels
pred_train_class=np.argmax(pred_train, axis=1)+1
pred_train_class=list(map(lambda x: str(x), pred_train_class))
pred_val_class=np.argmax(pred_val, axis=1)+1
pred_val_class=list(map(lambda x: str(x), pred_val_class))
#pred_test_class=np.argmax(pred_test, axis=1)+1

In [ ]:
true_train_class=np.argmax(Y_train, axis=1)+1
true_train_class=list(map(lambda x: str(x), true_train_class))
true_val_class=np.argmax(Y_val, axis=1)+1
true_val_class=list(map(lambda x: str(x), true_val_class))

In [ ]:
labels = [str(i) for i in range(1,16)]
cm_train = confusion_matrix(true_train_class, pred_train_class , labels)
#cm_train=cm_train.astype('float') / cm_train.sum(axis=1)[:, np.newaxis] #For normalizing
cm_val = confusion_matrix(true_val_class, pred_val_class , labels)
#cm_val=cm_val.astype('float') / cm_val.sum(axis=1)[:, np.newaxis] #For normalizing

In [ ]:
print_cm(cm_train, labels)

In [ ]:
print_cm(cm_val, labels)

In [ ]:
#Evaluation metrics for valdation dataset
print(metrics.classification_report(true_val_class, pred_val_class))